In [15]:
import numpy as np
import pandas as pd
import re

file_path = 'artvis_dump_NEW.csv'
data = pd.read_csv(file_path)
data

,a.id,a.firstname,a.lastname,a.gender,a.birthdate,a.deathdate,a.birthplace,a.deathplace,a.nationality,e.id,e.title,e.venue,e.startdate,e.type,e.paintings,e.country,e.city,e.latitude,e.longitude
0,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,489,"Second Post-Impressionist Exhibition. British,...",Grafton Galleries,1912,group,4,GB,London,51.514248,-0.093145
1,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,618,Exhibition of the Camden Town Group and Others,Public Art Galleries,1913,group,6,GB,Brighton,50.833333,-0.150000
2,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,720,The Second Exhibition of Works by Members of t...,Goupil Gallery,1915,group,4,GB,London,51.514248,-0.093145
3,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,729,Third Exhibition of Works by Members of the Lo...,Goupil Gallery,1915,group,5,GB,London,51.514248,-0.093145
4,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,650,The First Exhibition of Works by Members of Th...,Goupil Gallery,1914,group,5,GB,London,51.514248,-0.093145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72073,13998,Adam Emory,Albright,M,1862-01-01,1957-01-01,Monroe,Warrenville,US,224,Thirteenth Annual Exhibition,Carnegie Institute,1909,group,1,US,Pittsburgh,40.433300,-79.983300
72074,13999,Anna,Davidson,F,0000-01-01,0000-01-01,NaN,NaN,NaN,666,Erste Ausstellung der Freien Secession Berlin,Ausstellungshaus am Kurfürstendamm 208/9,1914,group,1,DE,Berlin,52.516667,13.400000
72075,13999,Anna,Davidson,F,0000-01-01,0000-01-01,NaN,NaN,NaN,547,Frühjahr-Ausstellung der Münchener Secession,Königliches Kunstausstellungsgebäude am Königs...,1913,group,1,DE,Munich,48.150000,11.583333
72076,13999,Anna,Davidson,F,0000-01-01,0000-01-01,NaN,NaN,NaN,547,Frühjahr-Ausstellung der Münchener Secession,"Verein bildender Künstler Münchens ""Secession""",1913,group,1,DE,Munich,48.150000,11.583333


In [16]:
# In case we want to focus only on Europe:
european_countries = [
    "GB", "FR", "DE", "IT", "ES", "CH", "BE", "NL", "SE", "NO", "FI", "DK", 
    "AT", "IE", "PT", "PL", "HU", "CZ", "SK", "EE", "LV", "LT", "LU", "SI", 
    "HR", "BG", "RO", "GR", "IS", "MT", "CY"
]
european_data = data[data['e.country'].isin(european_countries)]
data = european_data

In [17]:
artists = data[['a.id', 'a.firstname', 'a.lastname', 'a.gender', 'a.birthdate', 'a.deathdate', 'a.birthplace', 'a.deathplace', 'a.nationality']].drop_duplicates()
# artists.to_csv('artists.csv', index=False)
artists

,a.id,a.firstname,a.lastname,a.gender,a.birthdate,a.deathdate,a.birthplace,a.deathplace,a.nationality
0,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB
7,2,Peter,Alma,M,1886-01-01,1969-01-01,Medan,Amsterdam,NL
20,3,Natan,Alt'man,M,1889-01-01,1970-01-01,Vinnitsa,Saint Petersburg,RU
26,4,Cuno,Amiet,M,1868-01-01,1961-01-01,Solothurn,Seeberg,CH
57,5,Vassily,Kandinsky,M,1866-01-01,1944-01-01,Moscow,Neuilly-sur-Seine,RU
...,...,...,...,...,...,...,...,...,...
72048,13975,Otto von,Bylandt,M,1852-01-01,1929-01-01,"Hague, The",Versailles,NL
72050,13976,Max,Cramer,M,1882-01-01,1960-01-01,Bremen,"Reinach, Aargau",DE
72052,13977,George Laurens,Kiers,M,1838-01-01,1916-01-01,Amsterdam,Amsterdam,NL
72054,13979,Marinus Cornelis Thomas,Vermeulen,M,1868-01-01,1941-01-01,Tilburg,Amsterdam,NL


In [18]:
artist_id_count = artists['a.id'].value_counts().sort_values(ascending=False)
if len(artist_id_count[artist_id_count > 1]) > 0:
    print('Some artist IDs have duplicates')
else:
    print('No duplicate artist IDs')

No duplicate artist IDs


In [19]:
exhibitions = data[['e.id', 'e.title', 'e.venue', 'e.startdate', 'e.type', 'e.country', 'e.city', 'e.latitude', 'e.longitude']].drop_duplicates().sort_values('e.id')
exhibitions

,e.id,e.title,e.venue,e.startdate,e.type,e.country,e.city,e.latitude,e.longitude
108,3,2. Deutsche Künstlerbund Ausstellung,Deutscher Künstlerbund,1905,group,DE,-,51.500000,10.500000
115,3,2. Deutsche Künstlerbund Ausstellung,Ausstellungshaus am Kurfürstendamm 208/9,1905,group,DE,Berlin,52.516667,13.400000
7734,5,Exposition de Peintures et Dessins,Galerie B. Weill,1905,group,FR,Paris,48.866667,2.333333
5596,7,Edvard Munch. XV. výstava Spolku výtvarn. uměl...,Spolek výtvarných umělců Mánes,1905,solo,CZ,Prague,50.100000,14.433300
5574,7,Edvard Munch. XV. výstava Spolku výtvarn. uměl...,Pavilon v zahradě Kinských,1905,solo,CZ,Prague,50.100000,14.433300
...,...,...,...,...,...,...,...,...,...
917,1474,Союз Молодежи,Riga (exact location unknown),1910,group,LV,Riga,56.950000,24.100000
920,1475,"Выставка ""Импрессионистов"".",Vilnius,1909,group,LT,Vilnius,54.683300,25.316700
5553,1478,[Sonderausstellung Munchs im Bremer Kunstverei...,Kunsthalle Bremen,1905,group,DE,Bremen,53.083333,8.800000
51,1483,Ausstellung des Verbandes der Kunstfreunde in ...,Kunsthaus Zürich,1911,group,CH,Zurich,47.366667,8.550000


As we can see in the table above, for an exhibition ID we may have multiple entries. This means that many exhibitions come up more than once with a different venue, possibly because 
- the venue has multiple names or spelling, 
- the city is given for the venue name with the additional information 'exact location unknown',
- the association of artists is given as a venue instead of the building,
- a venue owner/committee/association organises an exhibition at another venue

In [20]:
e_counts = exhibitions['e.id'].value_counts()
#display(e_counts)
print(f'There are {len(e_counts)} different exhibition IDs.')
duplicate_exhibitions_counts = e_counts[e_counts > 1]
print(f'of which {len(duplicate_exhibitions_counts)} come up more than once for a given title, startdate and type.')
duplicate_exhibitions = exhibitions[exhibitions['e.id'].isin(duplicate_exhibitions_counts.index)].sort_values('e.id')
display(duplicate_exhibitions)
# exhibitions.to_csv('exhibitions.csv', index=False)

There are 979 different exhibition IDs.
of which 335 come up more than once for a given title, startdate and type.


,e.id,e.title,e.venue,e.startdate,e.type,e.country,e.city,e.latitude,e.longitude
108,3,2. Deutsche Künstlerbund Ausstellung,Deutscher Künstlerbund,1905,group,DE,-,51.500000,10.500000
115,3,2. Deutsche Künstlerbund Ausstellung,Ausstellungshaus am Kurfürstendamm 208/9,1905,group,DE,Berlin,52.516667,13.400000
5596,7,Edvard Munch. XV. výstava Spolku výtvarn. uměl...,Spolek výtvarných umělců Mánes,1905,solo,CZ,Prague,50.100000,14.433300
5574,7,Edvard Munch. XV. výstava Spolku výtvarn. uměl...,Pavilon v zahradě Kinských,1905,solo,CZ,Prague,50.100000,14.433300
2125,10,La Libre Esthétique. La douzième Exposition,Brussels (exact location unknown),1905,group,BE,Brussels,50.833333,4.333333
...,...,...,...,...,...,...,...,...,...
42459,1433,XLII. Ausstellung der Vereinigung bildender Kü...,Gebäude der Secession,1912,group,AT,Vienna,48.216700,16.366700
11313,1462,Les Indépendants Cercle d'Art. Salon annuel,Brussels (exact location unknown),1905,group,BE,Brussels,50.833333,4.333333
11314,1462,Les Indépendants Cercle d'Art. Salon annuel,Les Indépendants Cercle d'Art,1905,group,BE,Brussels,50.833333,4.333333
630,1465,VIII. Serie Französische Impressionisten,Kunsthaus Zürich,1908,group,CH,Zurich,47.366667,8.550000


We can see in the following table that in the artvis file, artists exhibiting in such exhibitions with duplicates are counted more than once for the exhibition, so we need to remove all those duplicate rows in order not to count mulltiple times their number of exhibitions or exhibited paintings.

In [21]:
artvis_multiple_counts = data[data['e.id'].isin(duplicate_exhibitions['e.id'])]
artvis_multiple_counts

,a.id,a.firstname,a.lastname,a.gender,a.birthdate,a.deathdate,a.birthplace,a.deathplace,a.nationality,e.id,e.title,e.venue,e.startdate,e.type,e.paintings,e.country,e.city,e.latitude,e.longitude
8,2,Peter,Alma,M,1886-01-01,1969-01-01,Medan,Amsterdam,NL,490,Moderne Kunst Kring (Cercle de l'art moderne)....,Moderne Kunst Kring,1912,group,13,NL,Amsterdam,52.350000,4.916667
9,2,Peter,Alma,M,1886-01-01,1969-01-01,Medan,Amsterdam,NL,609,Moderne Kunst Kring (Cercle de l'Art Moderne),Moderne Kunst Kring,1913,group,12,NL,Amsterdam,52.350000,4.916667
10,2,Peter,Alma,M,1886-01-01,1969-01-01,Medan,Amsterdam,NL,462,Internationale Kunstausstellung des Sonderbund...,Städtische Ausstellungshalle am Aachener Tor,1912,group,1,DE,Cologne,50.933333,6.950000
11,2,Peter,Alma,M,1886-01-01,1969-01-01,Medan,Amsterdam,NL,462,Internationale Kunstausstellung des Sonderbund...,Sonderbund Westdeutscher Kunstfreunde und Küns...,1912,group,1,DE,Cologne,50.933333,6.950000
12,2,Peter,Alma,M,1886-01-01,1969-01-01,Medan,Amsterdam,NL,490,Moderne Kunst Kring (Cercle de l'art moderne)....,Stedelijk Museum,1912,group,13,NL,Amsterdam,52.350000,4.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72053,13977,George Laurens,Kiers,M,1838-01-01,1916-01-01,Amsterdam,Amsterdam,NL,1397,Tentoonstelling van Teekeningen en Beeldhouwwe...,Amsterdam (exact location unknown),1909,group,2,NL,Amsterdam,52.350000,4.916667
72054,13979,Marinus Cornelis Thomas,Vermeulen,M,1868-01-01,1941-01-01,Tilburg,Amsterdam,NL,1397,Tentoonstelling van Teekeningen en Beeldhouwwe...,Amsterdam (exact location unknown),1909,group,1,NL,Amsterdam,52.350000,4.916667
72055,13979,Marinus Cornelis Thomas,Vermeulen,M,1868-01-01,1941-01-01,Tilburg,Amsterdam,NL,1397,Tentoonstelling van Teekeningen en Beeldhouwwe...,Arti et Amicitiae,1909,group,1,NL,Amsterdam,52.350000,4.916667
72075,13999,Anna,Davidson,F,0000-01-01,0000-01-01,NaN,NaN,NaN,547,Frühjahr-Ausstellung der Münchener Secession,Königliches Kunstausstellungsgebäude am Königs...,1913,group,1,DE,Munich,48.150000,11.583333


In order to get rid of those duplicate rows, we will group by the pair ('a.id', 'e.id') and keep only one row for each group. For this we need to define a function which tells which row of each group to keep.

In [22]:
# for multiple entries, we favor the ones which do not contain the following words in the venue column, because those words refer to associations of artists and are often used as a replacement for unknown venues.
def choose_from_duplicates(group):
    exclude_substrings = ['(exact location unknown)', 'Vereeniging', 'Vereinigung', 'Société', 'Künstlerbund', 'Verein', 'Verband', 'Association']
    pattern = '|'.join(map(re.escape, exclude_substrings))
    filtered_group = group[~group['e.venue'].str.contains(pattern, case=False, na=False, regex=True)]
    if not filtered_group.empty:
       return filtered_group.iloc[0]
    else:
       return group.iloc[0]

The following dataframe $df$ is the cleaned data and will be used from now on as source data.

In [26]:
# df = data.groupby(['a.id', 'e.id']).apply(choose_from_duplicates, include_groups=True).reset_index(drop=True)
# df = data.groupby(['a.id', 'e.id']).apply(lambda group: choose_from_duplicates(group, include_groups=True)).reset_index(drop=True)

def choose_from_duplicates(group, include_groups=False):
    # Implement handling for include_groups if needed
    if include_groups:
        # Example logic when include_groups=True
        print("Processing group with include_groups=True")
    return group.head(1)  # Example logic

df = data.groupby(['a.id', 'e.id']).apply(lambda group: choose_from_duplicates(group, include_groups=True)).reset_index(drop=True)

Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_groups=True
Processing group with include_grou

In [27]:
df

,a.id,a.firstname,a.lastname,a.gender,a.birthdate,a.deathdate,a.birthplace,a.deathplace,a.nationality,e.id,e.title,e.venue,e.startdate,e.type,e.paintings,e.country,e.city,e.latitude,e.longitude
0,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,489,"Second Post-Impressionist Exhibition. British,...",Grafton Galleries,1912,group,4,GB,London,51.514248,-0.093145
1,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,618,Exhibition of the Camden Town Group and Others,Public Art Galleries,1913,group,6,GB,Brighton,50.833333,-0.150000
2,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,650,The First Exhibition of Works by Members of Th...,Goupil Gallery,1914,group,5,GB,London,51.514248,-0.093145
3,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,680,Twentieth Century Art. A Review of Modern Move...,Whitechapel Art Gallery,1914,group,3,GB,London,51.514248,-0.093145
4,1,William Bernard,Adeney,M,1878-01-01,1966-01-01,London,London,GB,720,The Second Exhibition of Works by Members of t...,Goupil Gallery,1915,group,4,GB,London,51.514248,-0.093145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38563,13977,George Laurens,Kiers,M,1838-01-01,1916-01-01,Amsterdam,Amsterdam,NL,1397,Tentoonstelling van Teekeningen en Beeldhouwwe...,Arti et Amicitiae,1909,group,2,NL,Amsterdam,52.350000,4.916667
38564,13979,Marinus Cornelis Thomas,Vermeulen,M,1868-01-01,1941-01-01,Tilburg,Amsterdam,NL,1397,Tentoonstelling van Teekeningen en Beeldhouwwe...,Amsterdam (exact location unknown),1909,group,1,NL,Amsterdam,52.350000,4.916667
38565,13999,Anna,Davidson,F,0000-01-01,0000-01-01,NaN,NaN,NaN,547,Frühjahr-Ausstellung der Münchener Secession,Königliches Kunstausstellungsgebäude am Königs...,1913,group,1,DE,Munich,48.150000,11.583333
38566,13999,Anna,Davidson,F,0000-01-01,0000-01-01,NaN,NaN,NaN,573,Ausstellung des Deutschen Künstlerbundes 1913,Kunsthalle Mannheim,1913,group,1,DE,Mannheim,49.488333,8.464722


In [28]:
df.to_csv('artVis_data_cleaned.csv', index=False)

In [29]:
venues = df[['e.venue']].drop_duplicates()
venues

,e.venue
0,Grafton Galleries
1,Public Art Galleries
2,Goupil Gallery
3,Whitechapel Art Gallery
5,Doré Gallery
...,...
26618,Capitole
26619,Ancienne Caisse d'épargne
26622,Ancien Archevêché
27568,New Association of Artists


# Queries:

### Artists by date and number of paintings
For each year of the time range, the artists who exhibited in that year are listed, sorted by decreasing number of exhibited paintings. \
 \
$\rightarrow \ \texttt{artists_by_date_and_nb_of_paintings.csv}$

In [30]:
artists_by_date_and_paintings = df.groupby(['e.startdate', 'a.id']).agg({'a.firstname': 'first', 'a.lastname':'first', 'e.paintings': 'sum'})
artists_by_date_and_paintings.columns = ['firstname', 'lastname', 'number_of_paintings']
artists_by_date_and_paintings.sort_values(['e.startdate', 'number_of_paintings'], ascending=[True, False], inplace=True)
display(artists_by_date_and_paintings)
artists_by_date_and_paintings.to_csv('artists_by_date_and_nb_of_paintings.csv')

firstname    lastname  number_of_paintings
e.startdate a.id                                                 
1902        276           Edvard       Munch                   29
1903        276           Edvard       Munch                   57
1904        276           Edvard       Munch                   42
            595         Heinrich       Zille                   31
            7857           Louis      Vallet                   13
...                          ...         ...                  ...
1915        12339           Aldo      Severi                    1
            12340          Mario     Puccini                    1
            12341  Gaetano Mario  Parpagnoli                    1
            12342     Alessandro   Zenatello                    1
            12343      Raffaello       Ferro                    1

[28181 rows x 3 columns]

### Artists by date and number of venues
Again for each year of the time range, the artists are listed, sorted by decreasing number of venues at which they exhibited in that year. \
 \
$\rightarrow \ \texttt{artists_by_date_and_nb_of_venues.csv}$

In [31]:
artists_by_date_and_nb_of_venues = df.groupby(['e.startdate', 'a.id']).agg({'a.firstname': 'first', 'a.lastname':'first', 'e.venue': lambda x: x.nunique()})
artists_by_date_and_nb_of_venues.columns = ['firstname', 'lastname', 'number_of_venues']
artists_by_date_and_paintings.sort_values(['e.startdate'], ascending=True, inplace=True)
display(artists_by_date_and_nb_of_venues)
artists_by_date_and_nb_of_venues.to_csv('artists_by_date_and_nb_of_venues.csv')

firstname    lastname  number_of_venues
e.startdate a.id                                               
1902        276            Edvard       Munch                 2
1903        276            Edvard       Munch                 3
1904        267             Oskar        Moll                 1
            276            Edvard       Munch                 5
            317    Pierre-Auguste      Renoir                 1
...                           ...         ...               ...
1915        12341   Gaetano Mario  Parpagnoli                 1
            12342      Alessandro   Zenatello                 1
            12343       Raffaello       Ferro                 1
            12382        Benjamin       Coria                 1
            12848         Philipp      Helmer                 1

[28181 rows x 3 columns]

### Artists by date and number of exhibitions
For each year of the time range, the artists are listed, sorted by decreasing number of exhibitions they participated to that year. \
 \
$\rightarrow \ \texttt{artists_by_date_and_nb_of_exhibitions.csv}$

In [32]:
artists_by_date_and_nb_of_exhibitions = df.groupby(['e.startdate', 'a.id']).agg({'a.firstname': 'first', 'a.lastname':'first', 'e.title': 'count'})
artists_by_date_and_nb_of_exhibitions.columns = ['firstname', 'lastname', 'number_of_exhibitions']
artists_by_date_and_nb_of_exhibitions.sort_values(['e.startdate', 'number_of_exhibitions'], ascending=[True, False], inplace=True)
display(artists_by_date_and_nb_of_exhibitions)
artists_by_date_and_nb_of_exhibitions.to_csv('artists_by_date_and_nb_of_exhibitions.csv')

firstname    lastname  number_of_exhibitions
e.startdate a.id                                                    
1902        276            Edvard       Munch                      2
1903        276            Edvard       Munch                      4
1904        276            Edvard       Munch                      5
            267             Oskar        Moll                      1
            317    Pierre-Auguste      Renoir                      1
...                           ...         ...                    ...
1915        12341   Gaetano Mario  Parpagnoli                      1
            12342      Alessandro   Zenatello                      1
            12343       Raffaello       Ferro                      1
            12382        Benjamin       Coria                      1
            12848         Philipp      Helmer                      1

[28181 rows x 3 columns]

### Venues by date with number of artists, number of exhibitions and number of paintings
$\rightarrow \ \texttt{venues_by_date_nb_artists_nb_exhibitions_nb_paintings.csv}$

In [33]:
venues_by_date_nb_artists_nb_exhibitions_nb_paintings = df.groupby(['e.startdate', 'e.venue']).agg({'a.id': 'count', 'e.id':'count', 'e.paintings': 'sum', 'e.country': 'first', 'e.city':'first', 'e.latitude':'first', 'e.longitude':'first'})
venues_by_date_nb_artists_nb_exhibitions_nb_paintings.columns = ['nb_artists', 'nb_exhibitions', 'nb_paintings', 'country', 'city', 'latitude', 'longitude']
venues_by_date_nb_artists_nb_exhibitions_nb_paintings.sort_values(['e.startdate', 'nb_artists'], ascending=[True, False], inplace=True)
display(venues_by_date_nb_artists_nb_exhibitions_nb_paintings)
venues_by_date_nb_artists_nb_exhibitions_nb_paintings.to_csv('venues_by_date_nb_artists_nb_exhibitions_nb_paintings.csv')

nb_artists  \
e.startdate e.venue                                              
1902        Berliner Secession                               1   
            Kaiser Wilhelm Museum                            1   
1903        Paul Cassirer                                    2   
            Kunsthalle P.H.Beyer & Sohn                      1   
            Kunstsalon Paul Cassirer                         1   
...                                                        ...   
1915        Rotterdamsche Kunstkring                         3   
            Carfax Gallery                                   2   
            Annenstr. 30                                     1   
            Galleries of the Alpine Club                     1   
            Moderne Galerie (Heinrich Thannhauser)           1   

                                                    nb_exhibitions  \
e.startdate e.venue                                                  
1902        Berliner Secession                                   1   
            Kaiser Wilhelm Museum                                1   
1903        Paul Cassirer                                        2   
            Kunsthalle P.H.Beyer & Sohn                          1   
            Kunstsalon Paul Cassirer                             1   
...                                                            ...   
1915        Rotterdamsche Kunstkring                             3   
            Carfax Gallery                                       2   
            Annenstr. 30                                         1   
            Galleries of the Alpine Club                         1   
            Moderne Galerie (Heinrich Thannhauser)               1   

                                                    nb_paintings country  \
e.startdate e.venue                                                        
1902        Berliner Secession                                28      DE   
            Kaiser Wilhelm Museum                              1      DE   
1903        Paul Cassirer                                     31      DE   
            Kunsthalle P.H.Beyer & Sohn                       19      DE   
            Kunstsalon Paul Cassirer                           7      DE   
...                                                          ...     ...   
1915        Rotterdamsche Kunstkring                          64      NL   
            Carfax Gallery                                    80      GB   
            Annenstr. 30                                      21      DE   
            Galleries of the Alpine Club                      54      GB   
            Moderne Galerie (Heinrich Thannhauser)            67      DE   

                                                                        city  \
e.startdate e.venue                                                            
1902        Berliner Secession                                        Berlin   
            Kaiser Wilhelm Museum                                    Krefeld   
1903        Paul Cassirer                                             Berlin   
            Kunsthalle P.H.Beyer & Sohn                              Leipzig   
            Kunstsalon Paul Cassirer                                 Hamburg   
...                                                                      ...   
1915        Rotterdamsche Kunstkring                               Rotterdam   
            Carfax Gallery                                            London   
            Annenstr. 30                            Brandenburg an der Havel   
            Galleries of the Alpine Club                              London   
            Moderne Galerie (Heinrich Thannhauser)                    Munich   

                                                     latitude  longitude  
e.startdate e.venue                                                       
1902        Berliner Secession                      52.516667  13.400000  
            Kaiser Wilhelm 

# Functions to get top 10 artists and top 10 venues

In [34]:
def select_data_for_time_range(time_range_start, time_range_end):
    return df.loc[(df['e.startdate'] > time_range_start) & (df['e.startdate'] <= time_range_end), ['a.id', 'a.firstname', 'a.lastname', 'a.nationality', 'e.id', 'e.title', 'e.venue', 'e.startdate', 'e.type', 'e.paintings', 'e.country', 'e.city', 'e.latitude', 'e.longitude']]

In [35]:
def top_10_artists_by_paintings(time_range_start, time_range_end):
    result = select_data_for_time_range(time_range_start, time_range_end).groupby(['a.id']).agg({'a.firstname': 'first', 'a.lastname':'first', 'e.paintings': 'sum'})
    result.columns = ['firstname', 'lastname', 'number_of_paintings']
    result = result.sort_values('number_of_paintings', ascending=False).head(10)
    return result

display(top_10_artists_by_paintings(1910, 1914))

,firstname,lastname,number_of_paintings
a.id,,,
401,Vincent van,Gogh,682
164,Ferdinand,Hodler,480
276,Edvard,Munch,455
360,Walter Richard,Sickert,362
301,Pablo,Picasso,332
3665,Hanuš,Schwaiger,322
65,Lovis,Corinth,309
58,Paul,Cézanne,305
317,Pierre-Auguste,Renoir,286


In [36]:
def top_10_artists_by_exhibitions(time_range_start, time_range_end):
    result = select_data_for_time_range(time_range_start, time_range_end).groupby(['a.id']).agg({'a.firstname': 'first', 'a.lastname':'first', 'e.id': 'count'}) 
    result.columns = ['firstname', 'lastname', 'number_of_exhibitions']
    result = result.sort_values('number_of_exhibitions', ascending=False).head(10)
    return result

display(top_10_artists_by_exhibitions(1910, 1914))

,firstname,lastname,number_of_exhibitions
a.id,,,
317,Pierre-Auguste,Renoir,43
58,Paul,Cézanne,40
401,Vincent van,Gogh,33
1708,Camille,Pissarro,33
37,Pierre,Bonnard,32
225,Max,Liebermann,31
268,Claude,Monet,31
416,Edouard,Vuillard,30
164,Ferdinand,Hodler,28


In [37]:
def top_10_artists_by_venues(time_range_start, time_range_end):
    result = select_data_for_time_range(time_range_start, time_range_end).groupby(['a.id']).agg({'a.firstname': 'first', 'a.lastname':'first', 'e.venue': lambda x: x.nunique()}) 
    result.columns = ['firstname', 'lastname', 'number_of_venues']
    result = result.sort_values('number_of_venues', ascending=False).head(10)
    return result

display(top_10_artists_by_venues(1910, 1914))

,firstname,lastname,number_of_venues
a.id,,,
317,Pierre-Auguste,Renoir,23
401,Vincent van,Gogh,23
58,Paul,Cézanne,22
416,Edouard,Vuillard,21
37,Pierre,Bonnard,21
128,Othon,Friesz,20
301,Pablo,Picasso,20
83,Maurice,Denis,19
268,Claude,Monet,19


In [38]:
def top_10_venues_by_artists(time_range_start, time_range_end):
    result = select_data_for_time_range(time_range_start, time_range_end).groupby(['e.venue']).agg({'e.country': 'first', 'e.city':'first', 'e.latitude':'first', 'e.longitude':'first', 'a.id': lambda x: x.nunique()}) 
    result.columns = ['country', 'city', 'latitude', 'longitude', 'number_of_artists']
    result = result.sort_values('number_of_artists', ascending=False).head(10)
    return result

display(top_10_venues_by_artists(1910, 1914))

,country,city,latitude,longitude,number_of_artists
e.venue,,,,,
Stedelijk Museum,NL,Amsterdam,52.350000,4.916667,939
Exposition universelle de Gand,BE,Ghent,51.050000,3.716667,730
Königliches Kunstausstellungsgebäude am Königsplatz,DE,Munich,48.150000,11.583333,529
Ausstellungshaus am Kurfürstendamm 208/9,DE,Berlin,52.516667,13.400000,437
Città di Venezia,IT,Venice,45.438611,12.326667,422
Giardini Pubblici,IT,Venice,45.438611,12.326667,408
Művészház,HU,Budapest,47.500000,19.050000,378
Musée National du Luxembourg,FR,Paris,48.866667,2.333333,364
Palazzo dell'Esposizone,IT,Rome,41.900000,12.483333,292


In [39]:
def top_10_venues_by_exhibitions(time_range_start, time_range_end):
    result = select_data_for_time_range(time_range_start, time_range_end).groupby(['e.venue']).agg({'e.country': 'first', 'e.city':'first', 'e.latitude':'first', 'e.longitude':'first', 'e.id': lambda x: x.nunique()}) 
    result.columns = ['country', 'city', 'latitude', 'longitude', 'number_of_exhibitions']
    result = result.sort_values('number_of_exhibitions', ascending=False).head(10)
    return result

display(top_10_venues_by_exhibitions(1910, 1914))

,country,city,latitude,longitude,number_of_exhibitions
e.venue,,,,,
Galerie Druet,FR,Paris,48.866667,2.333333,38
Paul Cassirer,DE,Berlin,52.516667,13.400000,35
Carfax Gallery,GB,London,51.514248,-0.093145,27
MM. Bernheim-Jeune & Cie,FR,Paris,48.866667,2.333333,25
Der Sturm,DE,Berlin,52.516667,13.400000,23
Művészház,HU,Budapest,47.500000,19.050000,21
Galerie Miethke,AT,Vienna,48.216700,16.366700,18
Doré Gallery,GB,London,51.514248,-0.093145,16
Goupil Gallery,GB,London,51.514248,-0.093145,16


In [40]:
def top_10_venues_by_paintings(time_range_start, time_range_end):
    result = select_data_for_time_range(time_range_start, time_range_end).groupby(['e.venue']).agg({'e.country': 'first', 'e.city':'first', 'e.latitude':'first', 'e.longitude':'first', 'e.paintings': 'sum'}) 
    result.columns = ['country', 'city', 'latitude', 'longitude', 'number_of_paintings']
    result = result.sort_values('number_of_paintings', ascending=False).head(10)
    return result

display(top_10_venues_by_paintings(1910, 1914))

,country,city,latitude,longitude,number_of_paintings
e.venue,,,,,
Művészház,HU,Budapest,47.500000,19.050000,3670
Paul Cassirer,DE,Berlin,52.516667,13.400000,2779
Königliches Kunstausstellungsgebäude am Königsplatz,DE,Munich,48.150000,11.583333,2614
Stedelijk Museum,NL,Amsterdam,52.350000,4.916667,2372
Galerie Druet,FR,Paris,48.866667,2.333333,2095
Ausstellungshaus am Kurfürstendamm 208/9,DE,Berlin,52.516667,13.400000,1973
Città di Venezia,IT,Venice,45.438611,12.326667,1910
Giardini Pubblici,IT,Venice,45.438611,12.326667,1468
Carfax Gallery,GB,London,51.514248,-0.093145,1353
